In [5]:
import pickle
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
# import pandas as pd
import seaborn as sns
sns.set_theme()
sns.set_style("darkgrid")
# sns.set_context("paper")
# sns.set_context("talk")

In [145]:
with open('results/results_mlp.P', 'rb') as f:
    results = pickle.load(f)
cortical_results_mlp = results['Cortical']['analysis']
dist_results_mlp = cortical_results_mlp['dist_results']

with open('results/results_rnn.P', 'rb') as f:
    results = pickle.load(f)
cortical_results_rnn = results['Cortical']['analysis']
dist_results_rnn = cortical_results_rnn['dist_results']


In [146]:
reg_results = analyze_regression(dist_results_mlp, dist_results_rnn)

In [148]:
cat_regs = reg_results['cat_regs']
con_regs = reg_results['con_regs']
# categorical
param_cat_regs = cat_regs['params']
p_val_cat_regs = cat_regs['p_vals']
param_cat_regs.shape

(10, 210, 4)

In [ ]:
runs = param_cat_regs.shape[0]
checkpoints = param_cat_regs.shape[1]
ncoef = param_cat_regs.shape[2]
# continuous
param_con_regs = con_regs['params']
p_val_con_regs = con_regs['p_vals']

In [137]:
plot_regression_result(reg_results)

IndexError: tuple index out of range

In [133]:
def plot_regression_result(reg_results):
        cat_regs = reg_results['cat_regs']
        con_regs = reg_results['con_regs']
        # categorical
        param_cat_regs = cat_regs['params']
        p_val_cat_regs = cat_regs['p_vals']
        runs = param_cat_regs.shape[0]
        checkpoints = param_cat_regs.shape[1]
        ncoef = param_cat_regs.shape[2]
        # continuous
        param_con_regs = con_regs['params']
        p_val_con_regs = con_regs['p_vals']


        fig, axs = plt.subplots(2, ncoef, figsize=(20,5))
        for coef in range(ncoef):
            val_name = 'regression_x%s' %(coef)
            # categorical
            val = param_cat_regs[:,:,coef]
            df = pd.DataFrame(val, columns= np.arange(checkpoints))
            df.insert(0, 'runs', np.arange(runs))
            df2 = pd.melt(df, id_vars=['runs'],var_name='steps', value_name=val_name)
            # plot
            ax = axs[0][coef]
            ax = sns.boxplot(x='steps', y=val_name, data=df2, ax=ax)
            ax = sns.stripplot(x='steps', y=val_name, data=df2, ax=ax)
            for run in range(runs):
                for i, p in enumerate(p_val_cat_regs[run,:,coef]):
                    s = '*' if p<0.05 else ' '
                    ax.annotate(s, (i, param_cat_regs[run, i, coef]), color='r')
            # continuous
            val = param_con_regs[:,:,coef]
            df = pd.DataFrame(val, columns= np.arange(checkpoints))
            df.insert(0, 'runs', np.arange(runs))
            df2 = pd.melt(df, id_vars=['runs'],var_name='steps', value_name=val_name)
            # plot
            ax = axs[1][coef]
            ax = sns.boxplot(x='steps', y=val_name, data=df2, ax=ax)
            ax = sns.stripplot(x='steps', y=val_name, data=df2, ax=ax)
            for run in range(runs):
                for i, p in enumerate(p_val_con_regs[run,:,coef]):
                    s = '*' if p<0.05 else ' '
                    ax.annotate(s, (i, param_con_regs[run, i, coef]), color='r')

        return fig, axs


In [105]:
def calc_phi(grid_angles):
    phi = np.sin(2*grid_angles)
    binary_phi = np.sign(phi)
    for i, p in enumerate(phi):
        if np.abs(p)<1e-5:
            binary_phi[i] = 0
            
    return phi, binary_phi


In [152]:
    # hidd_dists_mlp = dist_results_mlp['hidd_dists']
    # grid_angles    = dist_results_mlp['grid_angles']
    # grid_dists     = dist_results_mlp['grid_dists']
    # hidd_dists_rnn = dist_results_rnn['hidd_dists']
    
    # runs = hidd_dists_mlp.shape[0]
    # checkpoints = hidd_dists_mlp.shape[1]
    # print(runs, checkpoints)
    

10 21


In [144]:
def analyze_regression(dist_mlp, dist_rnn):
    hidd_dists_mlp = dist_mlp['hidd_dists']
    grid_angles    = dist_mlp['grid_angles']
    grid_dists     = dist_mlp['grid_dists']
    hidd_dists_rnn = dist_rnn['hidd_dists']
    
    runs = hidd_dists_mlp.shape[0]
    checkpoints = hidd_dists_mlp.shape[1]
    
    
    p_val_cat_regs, t_val_cat_regs, param_cat_regs, bse_cat_regs = ([] for i in range(4))
    p_val_con_regs, t_val_con_regs, param_con_regs, bse_con_regs = ([] for i in range(4))

    for run in range(runs):
        p_val_cat_reg, t_val_cat_reg, param_cat_reg, bse_cat_reg = ([] for i in range(4))
        p_val_con_reg, t_val_con_reg, param_con_reg, bse_con_reg = ([] for i in range(4))
        for cp in range(checkpoints):
            hidd_dist_mlp = hidd_dists_mlp[run][cp]
            hidd_dist_rnn = hidd_dists_rnn[run][cp]
            grid_dist     = grid_dists[run][cp]
            grid_angle    = grid_angles[run][cp]
            phi, b_phi    = calc_phi(grid_angle)

            y = np.concatenate((hidd_dist_mlp, hidd_dist_rnn), axis=0)
            x1 = np.concatenate((grid_dist, grid_dist), axis=0)
            x3 = np.concatenate((np.ones(grid_dist.shape), -1*np.ones(grid_dist.shape)))

            x2_cat = np.concatenate((phi, phi), axis=0)
            x_cat = np.concatenate((x1.reshape((-1,1)), 
                                    x2_cat.reshape((-1,1)), 
                                    x3.reshape((-1,1))), axis=1)

            x2_con = np.concatenate((b_phi, b_phi), axis=0)
            x_con = np.concatenate((x1.reshape((-1,1)), 
                                    x2_con.reshape((-1,1)), 
                                    x3.reshape((-1,1))), axis=1)

            # categorical regression analysis
            x_cat = sm.add_constant(x_cat)
            stats_model_cat = sm.OLS(y,x_cat).fit()
            p_val_cat_reg.append(stats_model_cat.pvalues)
            t_val_cat_reg.append(stats_model_cat.tvalues)
            param_cat_reg.append(stats_model_cat.params)
            bse_cat_reg.append(stats_model_cat.bse)
            
            # continuous regression analysis
            x_con = sm.add_constant(x_con)
            stats_model_con = sm.OLS(y,x_con).fit()
            p_val_con_reg.append(stats_model_con.pvalues)
            t_val_con_reg.append(stats_model_con.tvalues)
            param_con_reg.append(stats_model_con.params)
            bse_con_reg.append(stats_model_con.bse)

        p_val_cat_regs.append(p_val_cat_reg)
        t_val_cat_regs.append(t_val_cat_reg)
        param_cat_regs.append(param_cat_reg)
        bse_cat_regs.append(bse_cat_reg)
        p_val_con_regs.append(p_val_con_reg)
        t_val_con_regs.append(t_val_con_reg)
        param_con_regs.append(param_con_reg)
        bse_con_regs.append(bse_con_reg)

    p_val_cat_regs = np.array(p_val_cat_regs)
    t_val_cat_regs = np.array(t_val_cat_regs)
    param_cat_regs = np.array(param_cat_regs)
    bse_cat_regs = np.array(bse_cat_regs)

    p_val_con_regs = np.array(p_val_con_regs)
    t_val_con_regs = np.array(t_val_con_regs)
    param_con_regs = np.array(param_con_regs)
    bse_con_regs = np.array(bse_con_regs)

    cat_regs = {'p_vals': p_val_cat_regs,
               't_vals': t_val_cat_regs,
               'params': param_cat_regs,
               'bses': bse_cat_regs}

    con_regs = {'p_vals': p_val_con_regs,
               't_vals': t_val_con_regs,
               'params': param_con_regs,
               'bses': bse_con_regs}

    reg_results = {'cat_regs': cat_regs, 
                   'con_regs': con_regs}
    return reg_results

